In [1]:
import sys
import logging
import os
import json
import pandas as pd

date_strftime_format = "%Y-%m-%y %H:%M:%S"
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s", datefmt=date_strftime_format)

# LOAD DATA

In [2]:
DATA_FOLDER = os.getcwd() + '/review_scorer/data/'
SENTI_PATH = DATA_FOLDER + 'SentiWord_info.json'
DATA_PATH = DATA_FOLDER + 'data_origin.csv'
data: pd.DataFrame

with open(SENTI_PATH, mode='rt', encoding='UTF8') as f:
    senti = pd.DataFrame.from_dict(json.load(f))

data = pd.read_csv(DATA_PATH, encoding='UTF8')
data = data.dropna(axis=0)
data = data.sample(frac=1).reset_index(drop=True)

# TOKENIZER

In [3]:
from tqdm import tqdm
from twkorean import TwitterKoreanProcessor

processor = TwitterKoreanProcessor()
tokenize = processor.tokenize_to_strings
tokens = [tokenize(_) for _ in tqdm(data.review)]

100%|████████████████████████████████████| 29557/29557 [03:16<00:00, 150.06it/s]


# SET REVIEW SCORER

In [7]:
from review_scorer import Doc2Category

# Review scorer needs tokens of datas to train when initializing it.
# 리뷰 채점기 클래스를 생성할 때, 토크나이즈 된 데이터를 인자로 주어야 합니다.
model = Doc2Category(sentences=tokens, senti_dict_path=SENTI_PATH)

2022-08-22 19:26:55 collecting all words and their counts
2022-08-22 19:26:55 PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-08-22 19:26:58 PROGRESS: at sentence #10000, processed 1140426 words, keeping 29957 word types
2022-08-22 19:27:01 PROGRESS: at sentence #20000, processed 2257599 words, keeping 42739 word types
2022-08-22 19:27:03 collected 51667 word types from a corpus of 3315605 raw words and 29557 sentences
2022-08-22 19:27:03 Creating a fresh vocabulary
2022-08-22 19:27:03 Doc2Category lifecycle event {'msg': 'effective_min_count=5 retains 15230 unique words (29.48% of original 51667, drops 36437)', 'datetime': '2022-08-22T19:27:03.936097', 'gensim': '4.2.0', 'python': '3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-65-generic-x86_64-with-debian-buster-sid', 'event': 'prepare_vocab'}
2022-08-22 19:27:03 Doc2Category lifecycle event {'msg': 'effective_min_count=5 leaves 3260723 word corpus 

## TAGGING RIVIEW SCORER's SENTIMENTAL DICTIONARY

In [8]:
# Tagging review scorer's sentimental dictionary by category.
# 카테고리에 따라 리뷰 채점기의 감성사전을 태깅합니다.
model.tag(categories={'taste': ['맛', '맛있다', '맛없다'],
                      'price': ['가격', '싸다', '비싸다', '저렴'],
                      'service': ['서비스', '친절', '싸가지'],
                      'atmosphere': ['인테리어', '분위기']},
          width=6, depth=6)

polarity: 100%|██████████████████████████| 14854/14854 [00:32<00:00, 458.94it/s]


# SCORING WITH REVIEW SCORER

In [9]:
start = 2000
for i in range(start, start + 10):
    print('index: ', i)
    print(model.score_review(tokenize(data.review.iloc[i])))
    print(data.review.iloc[i])
    print()

index:  2000
{'taste': 4.0, 'price': 0.5741626794258373, 'service': 0.0, 'atmosphere': 0.0}
순면 먹으러 여의도 정인면옥만두와 녹두전도 좋은데 순면이 최고

index:  2001
{'taste': 1.0, 'price': 1.278688524590164, 'service': 0.0, 'atmosphere': 0.0}
게가 들어있는 파스타 / 블랙 트러플 관자 리조또전경련 50층에 있어서 엘베탈때 귀가 아플수도 있습니다... 50층인만큼 뷰가 정말 끝내주고 예뻐요 ㅎㅎ 다만 음식 나오는 시간이 매우 오래걸렸습니다 ㅜㅜㅜ 컨셉도 이상해서 여자 직원들이 이상한 모자 쓰고있는데 안쓰러웠음....ㅠㅠㅠ 게가 들은 3만원짜리 파스타는 된장찌개 맛이 많이 났고, 블랙 트러플은 정말 맛있었음! 대신 계속 입술 검은색돼서 남친이랑 먹기는좀 그럴수돜ㅋㅋㅋㅋㅋ 가끔 분위기 낼때 가고싶습니다!

index:  2002
{'taste': 3.846547314578005, 'price': 1.8528512040160012, 'service': 0.0, 'atmosphere': 0.0}
맛있어요. 가성비 좋아요. 닭 한마리 치고 가격이 매우 비싼것 같지만 생각보다 매우 많아요. 3사람도 배불리 먹을 수 있어요

index:  2003
{'taste': 7.0, 'price': 2.8760838535610396, 'service': 0.0, 'atmosphere': 0.0}
증맬 자주간 식당입니드,,,우선 맛있어요팟타이 이렇게 만족스러운 곳 처음입니다 ㅜㅜ뿌팟퐁커리두 넘마싯구 ,, 쌀국수도 너무 좋아요메뉴 거의 다 먹어봤는데커리랑 팟타이가 제일 괜찮네요!!!근데 간이 느므 쎄요...!

index:  2004
{'taste': 4.0, 'price': 1.8528512040160012, 'service': 0.0, 'atmosphere': 0.0}
갈치가 들어가있는 김밥!! 완전 별미당 같이 파는 한치무침이랑 같이 

# SCORING REVIEWS

In [331]:
from sklearn.preprocessing import robust_scale

scores = pd.DataFrame(columns=model.category)
for i in tqdm(range(len(data)), desc='scoring'):
    scores.loc[i, :] = model.score_review(tokenize(data.review[i]))
scores = scores.astype('float')
scores = pd.DataFrame(robust_scale(scores, axis=1), columns=model.category)
data = pd.concat([data, scores], axis=1)
data.loc[:, model.category] = data.loc[:, model.category].astype('float')

scoring: 100%|████████████████████████████| 29557/29557 [25:16<00:00, 19.49it/s]


In [332]:
data

,Unnamed: 0,review,ID,title,load,가고싶다,평점,taste,price,service,atmosphere
0,6738,트러플 바오 ! 버섯을 싫어하는 사람도 정말 홀딱 반하게할 맛이다.집에 가서 이렇게...,해피뚜,바오바,서울특별시 용산구 녹사평대로40나길 1 2F,10473,4.4,0.451128,0.842105,-0.451128,-0.451128
1,8594,오픈 5분만에 만석이라니!! 이렇게 핫플인줄 모르고 운좋게 방문한 해운대 맛집 해목...,보람,해목,부산광역시 해운대구 구남로24번길 8,6031,4.5,1.462500,0.362500,-0.362500,-0.362500
2,549,"사이즈-닭 / 흑색 라멘 (13,000)웨이팅 25분, 그렇지만 기다릴 가치가 있는...",찻집고냉이,라무라,서울특별시 마포구 포은로 21 1F,3013,4.5,0.938666,0.425399,-0.425399,-0.508940
3,17590,(무나리뷰) 구서귀포에 위치한 전국구라는 군만두맛집. 예전부터 구서귀포에 군만두맛집...,무나&뎅,천일만두,제주특별자치도 서귀포시 서문로 25,283,4.4,1.319149,0.382979,-0.382979,-0.382979
4,7285,치미창가만 두접시 먹고 옴 - 정말 맛있다 !치미창가에 대한 호평이 많아서 먹어보고...,아지,와하카,서울특별시 성동구 성수일로12길 29,4851,4.4,0.498837,0.508138,-0.498837,-0.498837
...,...,...,...,...,...,...,...,...,...,...,...
29552,3473,가게이름 : #좋은소식⠀위치 : 수원시 영통구 매영로269번길 43(수인분당선 영통...,쿠리뷰,좋은소식,경기도 수원시 영통구 매영로269번길 43 3F 302호,2085,4.5,0.574966,0.489291,-0.489291,-0.489291
29553,22878,ㅠㅠ길게 쓴 리뷰가 오류로 날아갔네요..ㄸㄹㄹ요약정리하자면웨이팅을 40분이나 해서 ...,슈빙,맛이차이나,서울특별시 마포구 독막로 68 2F,7044,4.2,1.614360,0.340806,-0.340806,-0.340806
29554,13803,배부르게 잘먹었어요~~가격은 조금 사악하지만ㅜㅜ맛 하나만큼은 최고네요~~아침 일찍 ...,Eun❤,다츠,서울특별시 용산구 이태원로55나길 6 1F,4595,4.4,0.000000,4.000000,0.000000,0.000000
29555,15477,가격대비 별로지만 한번쯤 먹어볼만하다. 평양냉면 초보인 나에게 신기했던맛!,Dora,우래옥,서울특별시 중구 창경궁로 62-29,8473,4.4,1.333333,0.380952,-0.380952,-0.380952


# SCORE OF STORE & USERS

In [233]:
stores = data.title.drop_duplicates().copy()
users = data.ID.drop_duplicates().copy()
stores = pd.DataFrame(index=stores, columns=model.category)
users = pd.DataFrame(index=users, columns=model.category)

In [234]:
for store in tqdm(stores.index, desc='store'):
    stores.loc[store] = data.loc[data.title == store, model.category].mean()
stores = stores.astype('float')

for user in tqdm(users.index, desc='user'):
    users.loc[user] = data.loc[data.ID == user, model.category].mean()
users = users.astype('float')

user: 100%|████████████████████████████████| 6505/6505 [00:18<00:00, 358.24it/s]
